<a href="https://colab.research.google.com/github/renatojmf/Aspects-Extraction-in-Portuguese/blob/main/CAt_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-Processamento

In [ ]:
! pip install stanza

In [ ]:
! pip install spacy_conll

In [27]:
from google.colab import files
uploaded = files.upload()

Saving reli.csv to reli.csv


In [ ]:
import stanza as st
import pandas as pd
st.download('pt') # download pt model
nlp = st.Pipeline('pt') # initialize pt neural pipeline
df = pd.read_csv('/content/reli.csv',sep=';')

In [30]:
#passar apenas os textos/frases para a função
def getCoNLLU(df):
  dicts = []
  for texto in df:
    doc = nlp(texto) # doc is class Document
    dicts.append(doc.to_dict())

  return dicts

In [31]:
dicts = getCoNLLU(df['text'])

saida = ''
for sentenca in dicts[0:2]:
  for tokens in sentenca:
    for tks in tokens:
      for tag in tks:
        #print("tag:" ,tag)
        saida += str(tks[tag])  + "\t"
      saida +="\n"
    saida +="\n"
  saida +="\n"


print(saida)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


1	0	0	NUM	NumType=Card	0	root	0	1	


1	Ainda	ainda	ADV	5	cc	0	5	
2	assim	assim	ADV	1	fixed	6	11	
3	nao	nao	ADV	5	advmod	12	15	
4	me	eu	PRON	Case=Acc|Number=Sing|Person=1|PronType=Prs	5	obj	16	18	
5	apaixonei	apaixonar	VERB	Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin	0	root	19	28	
6	muito	muito	ADV	5	advmod	29	34	
7	por	por	ADP	9	case	35	38	
8	o	o	DET	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	9	det	39	40	
9	livro	livro	NOUN	Gender=Masc|Number=Sing	5	obl	41	46	
10	.	.	PUNCT	5	punct	47	48	





In [34]:
def getNouns():
  for sentenca in dicts:
    for tokens in sentenca:
      for tks in tokens:
        for tag in tks:
          if tks[tag] == 'NOUN':
            yield (tks['text'].lower())
          

In [35]:
from collections import Counter, defaultdict
import json

c = Counter()
c.update(getNouns())
nouns = getNouns()
print(c)

json.dump(dict(c), open('/content/nouns.json', 'w'))

Counter({'livro': 1243, 'historia': 419, 'leitura': 182, 'personagens': 150, 'romance': 112, 'personagem': 92, 'livros': 91, 'final': 83, 'obra': 81, 'vida': 80, 'mundo': 74, 'vezes': 72, 'narrativa': 70, 'tempo': 69, 'forma': 68, 'fim': 66, 'amor': 65, 'autor': 55, 'leitor': 54, 'comeco': 53, 'trama': 48, 'paginas': 45, 'anos': 44, 'coisas': 44, 'parte': 43, 'vampiros': 43, 'realidade': 41, 'atencao': 41, 'enredo': 41, 'filme': 41, 'inicio': 40, 'vampiro': 40, 'historias': 39, 'pessoas': 39, 'fato': 37, 'ponto': 33, 'coisa': 33, 'autora': 33, 'literatura': 32, 'verdade': 31, 'linguagem': 30, 'mae': 30, 'opiniao': 30, 'meninos': 29, 'pena': 29, 'sociedade': 29, 'meio': 29, 'protagonista': 29, 'gente': 27, 'familia': 27, 'saga': 27, 'maneira': 27, 'vez': 27, 'tipo': 26, 'momento': 25, 'duvida': 25, 'ideia': 25, 'acao': 25, 'suspense': 25, 'garota': 24, 'pai': 23, 'lado': 23, 'certeza': 23, 'momentos': 23, 'serie': 23, 'detalhes': 22, 'jeito': 22, 'palavras': 22, 'escrita': 22, 'partes':

In [122]:
def getCorpus2(df):
  corpus = []
  for item in df:
    corpus.append(item.split())
  
  return corpus

In [37]:
from gensim.models import Word2Vec
corpus = getCorpus2(df['text'])
#print(corpus[1::])

f = Word2Vec(corpus[1::],
                 sg=0,
                 negative=5,
                 window=10,
                 size=200,
                 min_count=2,
                 iter=5,
                 workers=10)

f.wv.save_word2vec_format("/content/my_word_vectors.vec")


In [ ]:
d = json.load(open('/content/nouns.json'))
nouns2 = Counter()
#print(f.wv.vocab.items[0])
for k, v in d.items():
  for tag,_ in f.wv.vocab.items():
    if k.lower() == tag:
      #print("valor v: ",v)
      nouns2[k.lower()] += v

# nouns2, _ = zip(*sorted(nouns2.items(),
#                 key=lambda x: x[1],
#                 reverse=True))

# json.dump(nouns, open("/content/aspect_words.json", "w"))

In [39]:
nouns3, _ = zip(*sorted(nouns2.items(),
                key=lambda x: x[1],
                reverse=True))

json.dump(nouns3, open("/content/aspect_words.json", "w"))

# Execução

In [ ]:
! pip install reach

In [20]:
import json
from reach import Reach
from collections import defaultdict
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter

In [3]:
def normalize(x):
    """Normalize a vector while controlling for zero vectors."""
    x = np.copy(x)
    if np.ndim(x) == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / np.linalg.norm(x)
    norm = np.linalg.norm(x, axis=-1)
    mask = norm > 0
    x[mask] /= norm[mask][:, None]
    return x


In [4]:
"""Simple method."""
import numpy as np
from collections import defaultdict
from sklearn.metrics.pairwise import rbf_kernel
from collections import Counter


def get_aspects(fragments, embeddings, n_adj_seed, n_nouns, min_count):
    """Get aspects based on fragments."""
    adj, _, noun = zip(*fragments)
    adj_cand, _ = zip(*Counter(adj).most_common(n_adj_seed))

    cands = candidate(embeddings,
                      adj,
                      noun,
                      adj_cand,
                      n_nouns,
                      min_count)

    return cands


def candidate(embeddings,
              adj,
              noun,
              seed_words,
              n_nouns,
              min_count):
    """
    Generates candidate aspects based on adjective co-occurrences

    Parameters
    ----------
    embeddings : Reach
        A Reach instance containing the word embeddings.
    constructions : list of tuples
        A list of adjective noun tuples.
    seed_words : list of str
        A list of strings. All these words should be in vocab for the
        given embeddings model.
    frequency_threshold : int
        Any noun occurring fewer times than this threshold is discarded
    n_nouns : int
        The amount of items to return

    Returns
    -------
    candidates : dict
        A dictionary mapping strings to their scores.

    """
    a = list(set(adj))
    sims = embeddings.similarity(a, seed_words).max(1)
    adj_scores = dict(zip(a, sims))

    noun_scores = defaultdict(lambda: [0, 0])
    for adj, noun in zip(adj, noun):
        noun_scores[noun][0] += adj_scores[adj]
        noun_scores[noun][1] += 1

    noun_scores = {k: v[0] for k, v in noun_scores.items()
                   if v[1] > min_count}

    return sorted(noun_scores.items(), key=lambda x: x[1])[-n_nouns:]


def rbf_attention(vec, memory, gamma, **kwargs):
    """
    Single-head attention using RBF kernel.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items
    gamma : float
        the gamma of the RBF kernel.

    Returns
    -------
    attention : np.array
        A (1, N)-shaped array, representing a single-headed attention mechanism

    """
    z = rbf_kernel(vec, memory, gamma)
    s = z.sum()
    if s == 0:
        # If s happens to be 0, back off to uniform
        return np.ones((1, len(vec))) / len(vec)
    return (z.sum(1) / s)[None, :]


def softmax(x, axis=1):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis, keepdims=True))
    s = e_x.sum(axis=axis, keepdims=True)
    return e_x / s


def attention(vec, memory, **kwargs):
    """
    Standard multi-head attention mechanism.

    Parameters
    ----------
    vec : np.array
        an (N, D)-shaped array, representing the tokens of an instance.
    memory : np.array
        an (M, D)-shaped array, representing the memory items

    Returns
    -------
    attention : np.array
        A (M, N)-shaped array, representing the attention over all memories.

    """
    z = memory.dot(vec.T)
    return softmax(z)


def mean(vec, aspect_vecs, **kwargs):
    """Just a mean weighting."""
    return (np.ones(len(vec)) / len(vec))[None, :]





In [5]:
def get_scores(instances,
               aspects,
               r,
               labels,
               remove_oov=False,
               attention_func=attention,
               **kwargs):
    """Scoring function."""
    assert all([x in r.items for x in labels])
    
    label_vecs = normalize(r.vectorize(labels))
    aspect_vecs = [x.mean(0)
                   for x in r.transform(aspects,
                                        remove_oov=False)]
    aspect_vecs = np.stack(aspect_vecs)
    if len(instances) == 1:
        instances = [instances]

    t = r.transform(instances, remove_oov=remove_oov)

    out = []
    for vec in t:
        att = attention_func(vec, aspect_vecs, **kwargs)
        # Att = (n_heads, n_words)
        z = att.dot(vec)
        # z = (n_heads, n_dim)
        x = normalize(z).dot(label_vecs.T)
        # x = (n_heads, n_labels)
        out.append(x.sum(0))
    return np.stack(out)

In [162]:
GAMMA = .03
N_ASPECT_WORDS = 200

scores = defaultdict(dict)
r = Reach.load("/content/my_word_vectors.vec",unk_word="<UNK>")

aspects = [[x] for x in json.load(open("/content/aspect_words.json"))]
aspects = aspects[:N_ASPECT_WORDS]

instances = ["Gostaria de um livro do homem-aranha".split(), "onde encontro o autor George RR Martin".split()]
label_set = {"historia", "livro", "autor"}

s = get_scores(instances,
               aspects,
               r,
               label_set,
               gamma=GAMMA,
               remove_oov=False,
               attention_func=rbf_attention)

pred = s.argmax(1)
print(s)
print(pred)

[[0.9999761  0.9999834  0.9999667 ]
 [0.99998045 0.99996144 0.9999836 ]]
[1 2]


# Funções auxiliares

In [161]:
with open('/content/restaurant_textos.txt ', 'w') as textfile:
  textfile.write("\n".join(str(item) for item in sentences))
textfile.close()

In [ ]:
with open('/content/corpus.txt ', 'w') as textfile:
  for item in corpus:
    textfile.write(str(item))
textfile.close()